# Chimera states

In this notebook we are going to inspect a variation of the Kuramoto model in which one can find chimera states. A chimera is a creature from greek mythology that is composed from different animals. In that sense a chimera state of a complex system is a state that is composed of different states. In particular we find that there is an existence of clustered ordered and unordered states at the same time.

In [1]:
from plotly import offline as py
from plotly import graph_objs as go

py.init_notebook_mode(connected=True)

We consider a ring of $N$ oscillators $\theta_1(t),\dots,\theta_N(t)$ with periodic boundary condition $\theta_{N+1}(t)=\theta_1(t)$. The index $i$ corresponds to $\phi_i=2\pi i/N-\pi$ on the unit circle. The dynamics are governed by, $$
\dot{\theta_i}(t)
=
-\frac{2\pi}{N}\sum^N_{j=1}G\left(\phi_i-\phi_j\right)\sin\left[\theta_i(t)-\theta_j(t)+\alpha\right],$$ with $$
G(x)
=
\frac{1+A\cos(x)}{2\pi}.$$

We initialize each oscillator according to $$\theta_i(0)=6u\exp\left[-0.76\phi_i^2\right],$$
wherein $u\sim\mathcal{U}\left(-\frac{1}{2},+\frac{1}{2}\right)$. The other parameters are set to:
* $A=0.995$
* $\alpha=\pi/2-0.18$
* $N=256$

In [255]:
from numba import jit

@jit
def kernel(x, A):
    return (1 + A * np.cos(x)) / (2 * np.pi)
@jit
def simulate(t, A, N, alpha, h):
    phi = np.linspace(-np.pi, +np.pi, N)
    time = np.arange(0, t, h)
    
    order = np.zeros([len(time), N], dtype=np.complex)
    theta = np.zeros([len(time), N])
    theta[0] = 6 * np.random.uniform(-0.5, +0.5, N) * np.exp(-0.76 * phi**2)

    for i in range(N):
        order[0, i] = 2 * np.pi * np.mean(kernel(phi[i] - phi, A) * np.exp(1j * theta[0]))
    
    for j in range(1, len(time)):
        for i in range(N):
            theta[j, i] = theta[j-1, i] + 2 * np.pi * np.mean(kernel(phi[i]-phi, A) * np.sin(theta[j-1,i] - theta[j-1] + alpha)) * h
            order[j, i] = 2 * np.pi * np.mean(kernel(phi[i] - phi, A) * np.exp(1j * theta[j]))    
        
    return phi, time, theta, order

In [260]:
phi1, time1, theta1, order1 = simulate(t=100, A=0.995, N=256, alpha=pi/2-0.18, h=0.001)
phi2, time2, theta2, order2 = simulate(t=100, A=0.1, N=256, alpha=pi/2-0.18, h=0.001)

In [261]:
layout = go.Layout(
    xaxis=dict(title='oscillator position (π)'),
    yaxis=dict(title='oscillator phase (π)'),
)

figure = go.Figure([
    go.Scatter(x=phi1 / np.pi, y=theta1[-1] / np.pi, mode='markers', name=f'A = 0.995'),
    go.Scatter(x=phi2 / np.pi, y=theta2[-1] / np.pi, mode='markers', name=f'A = 0.100'),
], layout)

py.iplot(figure)

In [262]:
layout = go.Layout(
    xaxis=dict(title='oscillator position (π)'),
    yaxis=dict(title='order parameter'),
)

figure = go.Figure([
    go.Scatter(x=phi1 / np.pi, y=np.abs(order1[-1]), mode='markers', name=f'A = 0.995'),
    go.Scatter(x=phi2 / np.pi, y=np.abs(order2[-1]), mode='markers', name=f'A = 0.100'),
], layout)

py.iplot(figure)

In [264]:
layout = go.Layout(
    xaxis=dict(title='oscillator position (π)'),
    yaxis=dict(title='mean phase (π)'),
)

figure = go.Figure([
    go.Scatter(x=phi1 / np.pi, y=np.angle(order1.mean(axis=0)) / np.pi, mode='markers', name=f'A = 0.995'),
    go.Scatter(x=phi2 / np.pi, y=np.angle(order2.mean(axis=0)) / np.pi, mode='markers', name=f'A = 0.100'),
], layout)

py.iplot(figure)